In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GroupKFold, cross_validate
import time

#RFRF
rf_param_grid = {
    "n_estimators": [50, 100, 200, 300],
    "max_depth": [10, 20, 30, None],
    "min_samples_split": [2, 5, 10],
    "min_samples_leaf": [1, 2, 4],
    "max_features": ["sqrt", "log2"],
    "class_weight": ["balanced", None],
}

total_combinations = (
    len(rf_param_grid["n_estimators"])
    * len(rf_param_grid["max_depth"])
    * len(rf_param_grid["min_samples_split"])
    * len(rf_param_grid["min_samples_leaf"])
    * len(rf_param_grid["max_features"])
    * len(rf_param_grid["class_weight"])
)

In [ ]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
scaler.fit(X_train)

X_train_s = scaler.transform(X_train)
X_val_s = scaler.transform(X_val)
X_test_s = scaler.transform(X_test)

print(f"train {X_train_s.shape} ({len(train_subjects)}")
print(f"val {X_val_s.shape} ({len(val_subjects)}")
print(f"test {X_test_s.shape} ({len(test_subjects)}")

In [ ]:
rf_param_grid_improved = {
    "n_estimators": [100, 200, 300],  
    "max_depth": [5, 10, 15, 20],  
    "min_samples_split": [10, 20, 50],  
    "min_samples_leaf": [5, 10, 20],  
    "max_features": ["sqrt", "log2"],  
    "class_weight": ["balanced", None],  
}

for param, values in rf_param_grid.items():
    print(f"  {param:20s}: {values}")

total_combinations_improved = (
    len(rf_param_grid_improved["n_estimators"])
    * len(rf_param_grid_improved["max_depth"])
    * len(rf_param_grid_improved["min_samples_split"])
    * len(rf_param_grid_improved["min_samples_leaf"])
    * len(rf_param_grid_improved["max_features"])
    * len(rf_param_grid_improved["class_weight"])
)

In [ ]:
n_cv_folds = min(5, len(np.unique(train_subjects)))
gkf = GroupKFold(n_splits=n_cv_folds)
train_groups = df_features.loc[train_mask, "subject"].values

print(f"folds: {n_cv_folds}")
print(f"training subjects: {sorted(train_subjects)}")
print(f"training samples: {len(X_train_s):,}")

In [ ]:
rf_cv_rows = []


total_iter = 0
start_time = time.time()

for n_est in rf_param_grid["n_estimators"]:
    for max_d in rf_param_grid["max_depth"]:
        for min_split in rf_param_grid["min_samples_split"]:
            for min_leaf in rf_param_grid["min_samples_leaf"]:
                for max_feat in rf_param_grid["max_features"]:
                    for class_w in rf_param_grid["class_weight"]:

                        total_iter += 1
                        if total_iter % 20 == 0 or total_iter == 1:
                            elapsed = time.time() - start_time
                            progress = total_iter / total_combinations * 100
                            eta = (elapsed / total_iter) * (
                                total_combinations - total_iter
                            )
                            
#---------------------------------#---------------------------------
                        rf = RandomForestClassifier(
                            n_estimators=n_est,
                            max_depth=max_d,
                            min_samples_split=min_split,
                            min_samples_leaf=min_leaf,
                            max_features=max_feat,
                            class_weight=class_w,
                            random_state=42,
                            n_jobs=-1,  
                        )

                        scores = cross_validate(
                            rf,
                            X_train_s, 
                            y_train,
                            groups=train_groups,
                            cv=gkf,
                            scoring=["accuracy", "f1_weighted"],
                            return_train_score=True,
                            n_jobs=-1,
                        )

                        rf_cv_rows.append(
                            {
                                "n_estimators": n_est,
                                "max_depth": max_d if max_d is not None else "None",
                                "min_samples_split": min_split,
                                "min_samples_leaf": min_leaf,
                                "max_features": max_feat,
                                "class_weight": (
                                    class_w if class_w is not None else "None"
                                ),
                                "cv_acc_mean": np.mean(scores["test_accuracy"]),
                                "cv_acc_std": np.std(scores["test_accuracy"]),
                                "cv_f1_mean": np.mean(scores["test_f1_weighted"]),
                                "cv_f1_std": np.std(scores["test_f1_weighted"]),
                                "tr_acc_mean": np.mean(scores["train_accuracy"]),
                                "tr_f1_mean": np.mean(scores["train_f1_weighted"]),
                            }
                        )

total_time = time.time() - start_time

In [ ]:
rf_cv_df = pd.DataFrame(rf_cv_rows).sort_values(
    ["cv_f1_mean", "cv_acc_mean"], ascending=False
)

print(rf_cv_df.head(10).to_string(index=False))

rf_best_row = rf_cv_df.iloc[0]
rf_best_params = {
    "n_estimators": int(rf_best_row["n_estimators"]),
    "max_depth": (
        None if rf_best_row["max_depth"] == "None" else int(rf_best_row["max_depth"])
    ),
    "min_samples_split": int(rf_best_row["min_samples_split"]),
    "min_samples_leaf": int(rf_best_row["min_samples_leaf"]),
    "max_features": rf_best_row["max_features"],
    "class_weight": (
        None if rf_best_row["class_weight"] == "None" else rf_best_row["class_weight"]
    ),
    "random_state": 42,
    "n_jobs": -1,
}

for param, value in rf_best_params.items():
    if param not in ["random_state", "n_jobs"]:
        print(f"  {param:20s}: {value}")


In [ ]:
X_trainval = np.vstack([X_train, X_val])
y_trainval = np.concatenate([y_train, y_val])


scaler_final = StandardScaler().fit(X_trainval)
X_trainval_s = scaler_final.transform(X_trainval)
X_test_s_rf = scaler_final.transform(X_test)
rf_final = RandomForestClassifier(**rf_best_params)

t0 = time.time()
rf_final.fit(X_trainval_s, y_trainval)
rf_train_time = time.time() - t0


print(f"no.trees: {rf_final.n_estimators}")
print(f"total features: {len(feature_columns)}")
print(f"features per split: {rf_final.max_features}")
print(f"model size: ~{rf_final.n_estimators * 50 / 1000:.1f}mb") 

In [ ]:
t0 = time.time()
rf_y_pred = rf_final.predict(X_test_s_rf)
rf_inference_time = time.time() - t0

# Calculate metrics
rf_acc = accuracy_score(y_test, rf_y_pred)
rf_prec = precision_score(y_test, rf_y_pred, average="weighted", zero_division=0)
rf_rec = recall_score(y_test, rf_y_pred, average="weighted", zero_division=0)
rf_f1 = f1_score(y_test, rf_y_pred, average="weighted", zero_division=0)


print(f"trees: {rf_best_params['n_estimators']}")
print(f"max depth: {rf_best_params['max_depth']}")
print(f"acc:  {rf_acc:.4f} ({rf_acc*100:.2f}%)")
print(f"precision: {rf_prec:.4f}")
print(f"recall: {rf_rec:.4f}")
print(f"f1: {rf_f1:.4f}")